In [1]:
import numpy as np

In [2]:
from random import randint

class Graph:
    """
    Graph data structure.
    """

    def __init__(self, fname = None, numVertices = None, numEdges = None, weightRange = None, directed = True):
        """
        Generates a weighted graph.
        """
        self.adjacent = {}
        self.weight = {}
        if fname == None:
            if any(arg == None for arg in (numVertices, numEdges, weightRange)):
                numVertices, numEdges, weightRange = map(int, input("numVertices, numEdges, weightRange: ").split())

            self.randomGraph(numVertices, numEdges, weightRange, directed)

        else:
            self.loadGraph(fname, directed)


    def numVertices(self):
        """
        Returns the number of vertices in the graph.
        """
        return len(self.adjacent)


    def vertices(self):
        """
        Returns the list of vertices in the graph.
        """
        return range(self.numVertices())


    def edges(self):
        """
        Returns a generator containing the edges in the graph.
        """
        return ((fromVertex,toVertex) for fromVertex in self.vertices() for toVertex in self.adjacent[fromVertex])


    def addDirectedEdge(self, fromVertex, toVertex, weight):
        """
        Inserts a weighted directed edge into the graph.
        """
        self.adjacent.setdefault(fromVertex, set()).add(toVertex)
        self.weight[(fromVertex, toVertex)] = weight


    def addUndirectedEdge(self, fromVertex, toVertex, weight):
        """
        Inserts a weighted undirected edge into the graph.
        """
        self.addDirectedEdge(fromVertex, toVertex, weight)
        self.addDirectedEdge(toVertex, fromVertex, weight)


    def randomGraph(self, numVertices, numEdges, weightRange, directed):
        """
        Generates a random graph.
        """
        addEdge = self.addDirectedEdge if directed else self.addUndirectedEdge

        for vertex in range(numVertices):
            self.adjacent[vertex] = set()

        for edge in range(numEdges):
            fromVertex = toVertex = None
            while fromVertex == toVertex:
                fromVertex = randint(0, numVertices-1)
                toVertex   = randint(0, numVertices-1)

            weight = randint(0, weightRange)
            addEdge(fromVertex, toVertex, weight)


    def loadGraph(fname, directed):
        """
        Loads a graph from a file containing a list of edges
        of the form: fromVertex, toVertex, weight.
        """ 
        addEdge = self.addDirectedEdge if directed else self.addUndirectedEdge
        with open(fname, 'r') as f:
            for vertex in range(int(f.readline())):
                self.adjacent[vertex] = set()

            for line in f.readlines():
                fromVertex, toVertex, weight = map(int, line.split())
                addEdge(fromVertex, toVertex, weight)


    def adjacentStr(self, fromVertex):
        """
        Returns a string representing the neighborhood of the
        given vertex.
        """
        return ", ".join(f"({toVertex}, {self.weight[(fromVertex, toVertex)]})" for toVertex in self.adjacent[fromVertex])


    def __str__(self):
        """
        Returns a string representing the graph.
        """
        return "\n".join(f"{vertex}: {self.adjacentStr(vertex)}" for vertex in range(self.numVertices()))


    def __repr__(self):
        """
        Represents the graph.
        """
        return str(self)


In [3]:
def APSP_DynamicProgramming(graph):
    """
    Applies dynamic programming to compute all-pairs
    shortest paths.
    """
    n = graph.numVertices()
    dst = [[float('inf') for v in range(n)] for u in range(n)]
    nxt = [[None for v in range(n)] for u in range(n)]

    for u in range(n):
        dst[u][u] = 0
        for v in graph.adjacent[u]:
            dst[u][v] = graph.weight[(u, v)]
            nxt[u][v] = v

    for m in range(1, n):
        for u in graph.vertices():
            for (x, v) in graph.edges():
                new_dst_uv = dst[u][x] + graph.weight[(x, v)]
                if new_dst_uv < dst[u][v]:
                    dst[u][v] = new_dst_uv
                    nxt[u][v] = nxt[u][x]


    # Check for negative weight cycles
    for u in graph.vertices():
        if any(dst[u][v] > dst[u][x] + graph.weight[(x,v)] for (x, v) in graph.edges()):
            return (None, None)

    return (dst, nxt)

In [4]:
def showResults(graph, dst, pointer):
    """
    Shows all-pairs shortest paths information.
    """
    if dst == None:
        print(None)
        return

    print("Distances:")
    for (v, row) in zip(graph.vertices(), dst):
        print(f"{v}: {row}")

    print("\nPath Pointers:")
    for (v, row) in zip(graph.vertices(), pointer):
        print(f"{v}: {row}")

In [6]:
graph = Graph()
print("\n\nDynamic Programming:")
print("-"*25)
dst, nxt = APSP_DynamicProgramming(graph)
showResults(graph, dst, nxt)



Dynamic Programming:
-------------------------
Distances:
0: [0, inf, inf, inf, inf, inf, 10, inf, inf, inf]
1: [inf, 0, inf, inf, inf, inf, inf, inf, inf, inf]
2: [7, 8, 0, inf, inf, inf, 0, inf, inf, inf]
3: [inf, 8, inf, 0, inf, inf, inf, inf, inf, inf]
4: [inf, inf, inf, inf, 0, inf, inf, inf, inf, inf]
5: [inf, inf, inf, inf, inf, 0, inf, inf, inf, inf]
6: [inf, inf, inf, inf, inf, inf, 0, inf, inf, inf]
7: [inf, inf, inf, inf, inf, inf, 2, 0, 0, inf]
8: [inf, inf, inf, inf, inf, inf, inf, inf, 0, inf]
9: [6, inf, inf, inf, inf, inf, 16, inf, inf, 0]

Path Pointers:
0: [None, None, None, None, None, None, 6, None, None, None]
1: [None, None, None, None, None, None, None, None, None, None]
2: [0, 1, None, None, None, None, 6, None, None, None]
3: [None, 1, None, None, None, None, None, None, None, None]
4: [None, None, None, None, None, None, None, None, None, None]
5: [None, None, None, None, None, None, None, None, None, None]
6: [None, None, None, None, None, None, None, None,